In [97]:
assignments = []

In [98]:
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

In [99]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    for u in unitlist:
        # lookup the values of the unitlist
        l_val=[values[s] for s in u]
        # count the number of duplicates in the values
        l_val_c=[l_val.count(s) for s in l_val]
        #find the naked twins boxes
        naked_twins=[u[i] for i in range(9) if l_val_c[i]==2 and len(l_val[i])==2]
        #find the non twin boxes
        non_twins=set(u)-set(naked_twins)
        #iterate all the naked_twins in case there are multiple pairs
        for box in naked_twins:
            for digit in values[box]:
                for n in non_twins:
                    if len(values[n])>1:
                        assign_value(values,n,values[n].replace(digit,''))
                        #values[n]=values[n].replace(digit,'')
        
    return values

In [100]:
def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

In [101]:
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))

In [102]:
def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [103]:
def eliminate(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            assign_value(values,peer,values[peer].replace(digit,''))
            #values[peer] = values[peer].replace(digit,'')
    return values

def only_choice(values):
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                assign_value(values,dplaces[0],digit)
                #values[dplaces[0]] = digit
    return values

In [104]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)
        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [105]:
def search(values):
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier in reduce_puzzle
    if all(len(values[s]) == 1 for s in values): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in values if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        assign_value(new_sudoku,s,value)
        #new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [106]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    values = search(grid_values(grid))
    #for box,value in values.items():
    #   assign_value(values,box,value)
    return values

In [107]:
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
# define diagonal units
diagonal_units = [[rows[i]+cols[i] for i in range(9)],[rows[i]+cols[-i-1] for i in range(9)]]
# add diagonal units to unitlist
unitlist = row_units + column_units + square_units + diagonal_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [ ]:
if __name__ == '__main__':
    diag_sudoku_grid = '9.1....8.8.5.7..4.2.4....6...7......5..............83.3..6......9................'
    display(solve(diag_sudoku_grid))

    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')

9 6 1 |3 5 4 |2 8 7 
8 3 5 |2 7 6 |1 4 9 
2 7 4 |8 9 1 |3 6 5 
------+------+------
4 2 7 |1 3 8 |9 5 6 
5 8 3 |9 6 2 |7 1 4 
6 1 9 |5 4 7 |8 3 2 
------+------+------
3 4 2 |6 8 9 |5 7 1 
7 9 8 |4 1 5 |6 2 3 
1 5 6 |7 2 3 |4 9 8 
